In [1]:
# Importation des librairies
import zipfile
import gzip
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from scipy.stats import norm

In [ ]:
gzip_path = r'H:\DATA\Datafiles\Projets\CPAM_Data\NTOT2024_12.CSV.gz'

# Importation des données et du fichier descriptif des variables
data = pd.read_csv(gzip_path, sep=';', compression='gzip', encoding='ISO-8859-1')

C:\Users\saeru\AppData\Local\Temp\ipykernel_23920\4165348403.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(gzip_path, sep=';', compression='gzip', encoding='ISO-8859-1')


In [8]:
data.head()

,PRS_FAC_TOP,l_fac_top,l_serie,SERIE,prs_nat,l_prs_nat,rem_date,sns_date,asu_nat,l_asu_nat,...,l_exe_spe,exe_spe1,l_exe_spe1,exe_stj1,l_exe_stj1,rem_mon,rec_mon,dep_mon,act_dnb,act_coe
0,0,0-Régime Général stricto-sensu,C Omnipraticiens,111111,1045,EXAM PARENTAL GROSSESSE CO-PARENT,202412,202411,3,Maternité,...,01-Médecine générale,11,Médecins Omnipraticiens,1,Libéral,"26,50","26,50","0,00",1.0,"1,00"
1,0,0-Régime Général stricto-sensu,C Omnipraticiens,111111,1045,EXAM PARENTAL GROSSESSE CO-PARENT,202412,202412,3,Maternité,...,01-Médecine générale,11,Médecins Omnipraticiens,1,Libéral,"26,50","26,50","0,00",1.0,"1,00"
2,0,0-Régime Général stricto-sensu,C Omnipraticiens,111111,1048,CONSULTATION INIT MT ALD,202409,202408,1,Maladie,...,01-Médecine générale,11,Médecins Omnipraticiens,1,Libéral,"120,00","120,00","0,00",2.0,"2,00"
3,0,0-Régime Général stricto-sensu,C Omnipraticiens,111111,1048,CONSULTATION INIT MT ALD,202409,202409,1,Maladie,...,01-Médecine générale,11,Médecins Omnipraticiens,1,Libéral,"1.812,00","1.812,00","0,00",30.0,"30,00"
4,0,0-Régime Général stricto-sensu,C Omnipraticiens,111111,1048,CONSULTATION INIT MT ALD,202410,202409,1,Maladie,...,01-Médecine générale,11,Médecins Omnipraticiens,1,Libéral,"60,00","60,00","0,00",1.0,"1,00"
